In [1]:
import pandas as pd
# pd.io.parquet.get_engine('auto')

from glob import glob

### Set tradable coins which has more than `$1500M` marketcap

In [2]:
with open('../rawdata/candidate_coins.txt', 'r') as f:
    candidate_coins = f.read().splitlines()

### Load file & store after cleaning

In [3]:
file_list = glob('../../../storage/dataset/rawdata/parquet/*.parquet')
file_list = [file for file in file_list if file.split('rawdata/parquet/')[-1].split('.parquet')[0] in candidate_coins]

In [4]:
from tqdm import tqdm

query_min_start_dt = '2018-06-01'

n_skipped = 0
for file in tqdm(file_list):
    df = pd.read_parquet(file)[['open', 'high', 'low', 'close', 'volume']]
    df = df.resample('1T').ffill()
    
    df = df[query_min_start_dt:]
    assert not df.isnull().any().any()

    csv_name = file.split('rawdata/parquet/')[-1].split('.parquet')[0] + '.csv'
    
    # skip exists data 
    if df.index[0] > pd.Timestamp(query_min_start_dt, freq='T'):
        print(f'skipped to save: {csv_name}')
        n_skipped += 1

        continue

    df.index = df.index.tz_localize('utc')
    df.to_csv('../../../storage/dataset/rawdata/csv/' + csv_name)

print(f'skipped: {n_skipped}')

0it [00:00, ?it/s]

skipped: 0
